In [ ]:
!pip install -q transformers==4.31.0 accelerate sentence-transformers datasets langchain faiss-cpu peft bitsandbytes loralib einops gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.7 MB/s eta 0:00

In [ ]:
from copy import deepcopy
from time import perf_counter, sleep

import gradio as gr
import numpy as np
import torch
from datasets import Dataset
from langchain.docstore.document import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from peft import PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    StoppingCriteria,
    StoppingCriteriaList,
    pipeline,
)

In [ ]:

# load embeddings
model_name = "keepitreal/vietnamese-sbert"
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}
embedder = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

In [ ]:
# load data
hcc_questions = Dataset.from_json(
    "/content/drive/MyDrive/Data/hcc_question_only.jsonl"
)["question"]
hcc_questions.extend(
    [
        "làm thủ tục hải quan đối với hàng hoá xuất nhập khẩu như thế nào?",
        "thủ tục đăng ký mới bảo hiểm xã hội đối với hàng hoá xuất nhập khẩu bao gồm những bước nào",
        "tôi có thể đến đâu để xin cấp đăng ký mới địa chỉ IP?",
        "cơ quan nào có thẩm quyền xử lý thủ tục xin cấp mới căn cước công dân",
        "điều kiện cần để đăng ký giấy chứng nhận thương binh là gì?",
        'thủ tục giải quyết hưởng chế độ ốm đau bao gồm những bước nào?',
        'có thể làm thủ tục cấp tạm trú tại cơ quan nào?',
        'tôi có thể nộp trực tiếp thủ tục đăng kí mới thương binh hay không?'
    ]
)

other_questions = Dataset.from_json("/content/drive/MyDrive/Data/temp.jsonl").shuffle(seed=42).select(range(1000))[
    "question"
]
data = [
    {"intent": "hcc", "questions": hcc_questions},
    {"intent": "other", "questions": other_questions},
    {
        "intent": "continue",
        "questions": [
            "Vậy thủ tục này đăng ký ở cơ quan nào",
            "thủ tục này do cơ quan nào cấp?",
            "Các bước thực hiện thủ tục này thì sao",
            "vậy còn trình tự thực hiện thủ tục này gồm những bước thế nào?",
            "cơ quan nào có trách nhiệm làm thủ tục này?",
            "Tôi có thể nộp thủ tục này trực tiếp hay trực tuyến?",
            "đối tượng thực hiện của văn bản này là gì?",
            "Hồ sơ để làm thủ tục này bao gồm những gì",
            "thủ tục trên thuộc lĩnh vực nào?",
            "tên chính xác của thủ tục hành chính ở trên là gì?",
            "Cơ quan nào được uỷ quyền thực hiện thủ tục ở trên?",
            "Có yêu cầu gì đặc biệt đối với thủ tục này hay không?",
            'thủ tục này có yêu cầu điều kiện thực hiện như thế nào?',
        ],
    },
    {
        "intent": "greeting",
        "questions": [
            "xin chào",
            "chào bạn",
            "xin chào, tôi là Hoàng",
            "hello",
            "hi",
            "chào thân ái",
            "rất vui được gặp bạn",
        ],
    },
    {
        "intent": "ability",
        "questions": [
            "bạn có thể giúp gì cho tôi",
            "bạn là ai?",
            "Mày là ai?",
            "bạn có thể giúp tôi giải đáp chủ đề hành chính công được không",
            "hãy giúp tôi về lĩnh vực hành chính công",
            "hãy giúp tôi",
            "hỗ trợ tao lĩnh vực hành chính công",
            "bạn có khả năng gì?",
            "mày có khả năng gì",
        ],
    },
    {
        "intent": "thanks",
        "questions": [
            "cảm ơn bạn",
            "bạn làm tốt lắm",
            "giỏi quá",
            "bạn đúng rồi",
            "mày trả lời tốt quá",
            "tuyệt vời",
            "cảm ơn bạn đã giúp tôi",
        ],
    },
    {
        "intent": "ending",
        "questions": [
            "tạm biệt",
            "hẹn gặp lại",
            "gặp lại bạn sau",
            "chào tạm biệt",
        ],
    },
]
for x in data:
  print(len(x['questions']))

972
1000
13
7
9
7
4


In [ ]:
# intent cls
class IntentClassifier:
    def __init__(self, embedder, data):
        self.data = data
        self.embedder = embedder
        self._build()

    def _build(self):
        temp, self.metadata = [], []
        for intent_type in self.data:
            for question in intent_type["questions"]:
                temp.append(question)
                self.metadata.append(intent_type["intent"])
        self.embeddings = self.embedder.embed_documents(temp)

    def __call__(self, question):
        question_repr = self.embedder.embed_query(question)
        scores = np.dot(self.embeddings, question_repr)
        return self.metadata[np.argmax(scores)]


intent_cls = IntentClassifier(embedder, data)

In [ ]:
intent_cls('vậy tôi muốn đăng ký thủ tục cấp lại giấy căn cước công dân thì sao?')

'hcc'

### Retriever

In [ ]:
corpus = Dataset.from_json("/content/drive/MyDrive/Data/hcc_data_simple.jsonl")
documents = [Document(page_content=x) for x in corpus["simple_text"]]
text_splitter = CharacterTextSplitter(chunk_size=1024, chunk_overlap=256)
texts = text_splitter.split_documents(documents)
db = FAISS.from_documents(texts, embedder)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.3, "k": 1},
)

def retrieve_func(text, retriever=retriever):
    result = retriever.get_relevant_documents(text)
    if len(result) == 0:
        return None
    return result[0].page_content

In [ ]:
retrieve_func('thủ tục cấp chứng nhận xuất xử hàng hoá ưu đãi bao gồm những bước nào')

'Mã thủ tục: 1.000586\nSố quyết định: 109/QĐ-NHNN\nTên thủ tục: Thủ tục đề nghị cấp Giấy chứng nhận đủ điều kiện sản xuất vàng trang sức, mỹ nghệ\nCấp thực hiện: Cấp Tỉnh\nLoại thủ tục: TTHC được luật giao quy định chi tiết\nLĩnh vực: Hoạt động ngoại hối\nTrình tự thực hiện: + Bước 1: Doanh nghiệp gửi hồ sơ đến Ngân hàng Nhà nước chi nhánh tỉnh, thành phố nơi doanh nghiệp đặt trụ sở chính.+ Bước 2: Ngân hàng Nhà nước chi nhánh tiến hành kiểm tra thực tế cơ sở vật chất, trang thiết bị phục vụ hoạt động sản xuất vàng trang sức, mỹ nghệ của doanh nghiệp.+ Bước 3: Căn cứ quy định về điều kiện, hồ sơ, thủ tục và kết quả kiểm tra thực tế cơ sở vật chất, trang thiết bị phục vụ hoạt động sản xuất vàng trang sức, mỹ nghệ của doanh nghiệp, trong thời hạn 30 ngày làm việc kể từ ngày nhận đủ hồ sơ hợp lệ, Ngân hàng Nhà nước chi nhánh xem xét, cấp hoặc từ chối cấp Giấy chứng nhận đủ điều kiện sản xuất vàng trang sức, mỹ nghệ.\nĐối tượng thực hiện: Doanh nghiệp\nCơ quan thực hiện: Ngân hàng Nhà nước

### Memory

In [ ]:
# Memory
class BaseMemoryConstrucor:
    def __init__(self, system_message, human_symbol, ai_symbol):
        self.system_message = system_message
        self.human_symbol = human_symbol
        self.ai_symbol = ai_symbol
        self.memory = []  # [(human_input, bot_response)]

    def get_full_conversation(self):
        conversation = self.system_message
        for human, ai in self.memory:
            conversation += (
                f"{self.human_symbol} {human}\n" + f"{self.ai_symbol} {ai}\n"
            )
        return conversation

    def clear_memory(self):
        self.memory = []

    def add_to_memory(self, human_input, ai_response):
        self.memory.append((human_input, ai_response))

    def pop_from_memory(self):
        if len(self.memory) > 0:
            self.memory.pop()

    def get_used_memory(self):
        pass

    def construct_input_memory(self, human_input):
        return (
            self.get_used_memory()
            + f"{self.human_symbol} {human_input}\n"
            + f"{self.ai_symbol} "
        )


class FixedWindowLengthMemoryConstructor(BaseMemoryConstrucor):
    def __init__(self, window_length, system_message, human_symbol, ai_symbol):
        super().__init__(system_message, human_symbol, ai_symbol)
        self.window_length = window_length

    def get_used_memory(self):
        conversation = self.system_message
        for human, ai in self.memory[-self.window_length :]:
            conversation += (
                f"{self.human_symbol} {human}\n" + f"{self.ai_symbol} {ai}\n"
            )
        return conversation

## Inferencer

In [ ]:
default_gen_config = {
    "temperature": 0.8,
    "top_p": 0.9,
    "top_k": 20,
    "max_new_tokens": 512,
}

class Inferencer:
    def __init__(
        self,
        adapter_path,
        base_model_path,
        gen_config=default_gen_config,
        tokenizer_max_length=1024,
        human_symbol="[|Con người|]",
    ):
        self.human_symbol = human_symbol
        self.gen_config = deepcopy(gen_config)
        self.model = None
        self.tokenizer = None
        self.pipeline = None
        self.stopping_criteria = None

        # need to run these functions in order
        print("Loading model and tokenizer...")
        self._load_model_and_tokenizer(
            adapter_path, base_model_path, tokenizer_max_length
        )
        print("Setting stopping criteria...")
        self._set_stopping_criteria([self.human_symbol, self.tokenizer.eos_token])
        print("Building generation pipeline...")
        self._build_pipeline()

    def _load_model_and_tokenizer(
        self, adapter_path, base_model_name_or_path, tokenizer_max_length
    ):
        model = AutoModelForCausalLM.from_pretrained(
            base_model_name_or_path,
            quantization_config=BitsAndBytesConfig(
                load_in_4bit=True,
                device_map="auto",
                bnb_4bit_quant_type="nf4",
                bnb_4bit_use_double_quant=True,
                bnb_4bit_compute_dtype=torch.float32,
            ),
            trust_remote_code=True,
            torch_dtype=torch.float32,
        )
        self.model = PeftModel.from_pretrained(model, adapter_path)
        self.tokenizer = AutoTokenizer.from_pretrained(base_model_name_or_path)
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.tokenizer.model_max_length = tokenizer_max_length

    def _set_stopping_criteria(self, stop_seq_list=[]):
        stop_token_ids_list = [
            torch.tensor(
                self.tokenizer.convert_tokens_to_ids(self.tokenizer.tokenize(x))
            )
            .long()
            .to("cuda")
            for x in stop_seq_list
        ]

        class StopOnTokens(StoppingCriteria):
            def __call__(
                self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs
            ) -> bool:
                for stop_ids in stop_token_ids_list:
                    if torch.eq(input_ids[0][-len(stop_ids) :], stop_ids).all():
                        return True
                return False

        self.stopping_criteria = StoppingCriteriaList([StopOnTokens()])

    def _build_pipeline(self):
        self.pipeline = pipeline(
            model=self.model,
            tokenizer=self.tokenizer,
            stopping_criteria=self.stopping_criteria,
            return_full_text=True,
            task="text-generation",
            **self.gen_config,
        )

    def set_gen_config(self, gen_config):
        self.gen_config = gen_config
        self._build_pipeline()

    def generate(self, prompt):
        start = perf_counter()
        text_output = self.pipeline(prompt)[0]["generated_text"]
        total_time = perf_counter() - start
        print(f"### Generated in {total_time:.6f} seconds ###\n")
        return text_output


In [ ]:
inferencer = Inferencer(
    adapter_path='hoang14/chatbot_29_8',
    base_model_path='bigscience/bloom-3b',
    gen_config=default_gen_config,
    tokenizer_max_length=1024,
)

Loading model and tokenizer...


Some weights of BloomForCausalLM were not initialized from the model checkpoint at bigscience/bloom-3b and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Setting stopping criteria...
Building generation pipeline...


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'O

In [ ]:
gen_func = lambda prompt: inferencer.generate(prompt)

### APP logic

In [ ]:
class HCCApp:
    def __init__(self, gen_func, retrieve_func, intent_cls_func, memory):
        self.intent_cls_func = intent_cls_func
        self.gen_func = gen_func
        self.retrieve_func = retrieve_func
        self.memory = memory
        self.main_context = None
        self.template_response = {
            "other": "Xin lỗi, tôi không thể trả lời câu hỏi này.",
            "thanks": "Rất vui vì điều đó, tôi có thể hỗ trợ gì thêm cho bạn không?",
            "greeting": "Xin chào, tôi là trợ lý ảo hành chính công. Tôi có thể giúp gì cho bạn?",
            "ability": "Tôi là trợ lý ảo của sếp Hoàng. Tôi có thể giúp bạn trả lời những thắc mắc về lĩnh vực hành chính công. Hãy bắt đầu bằng cách đưa ra những câu hỏi nhé!",
            "ending": "Cảm ơn bạn đã sử dụng dịch vụ của tôi. Hẹn gặp lại bạn sau!",
            "unclear": "Xin lỗi, tôi không hiểu câu hỏi của bạn. Hoặc bạn hãy đưa ra câu hỏi rõ ràng hơn.",
        }
        self.qa_template = """Cuộc trò chuyện giữa con người và trợ lý AI.
[|Con người|] Trả lời câu hỏi sau đây dựa vào thông tin nằm trong đoạn văn bản.
Câu hỏi: {question}
Đoạn văn bản:
{context}
Câu trả lời cho câu hỏi '{question}' là:
[|AI|] """

    def _return_answer(self, question, answer):
        self.memory.add_to_memory(question, answer)
        return answer

    def answer(self, question):
            intent = self.intent_cls_func(question)
            if intent in self.template_response:
                return self._return_answer(question, self.template_response[intent])

            if intent == "hcc":
                context = retrieve_func(question)
                if context is None:
                    return self._return_answer(question, self.template_response["unclear"])
                self.main_context = context

            if intent == "continue":
                if self.main_context is None:
                    return self._return_answer(question, self.template_response["unclear"])

            prompt = self.qa_template.format(question=question, context=self.main_context)
            answer = (
                self.gen_func(prompt)
                .split("[|AI|]")[1]
                .strip()
                .split("[|Con người|]")[0]
                .strip()
            )
            return self._return_answer(question, answer)

    def render(self, share=True):
        def chat(human_input, history=self.memory.memory):
            yield "", history + [(human_input, None)]
            response = ""
            for word in self.answer(human_input).split(" "):
                sleep(0.05)
                response += word + " "
                yield "", history + [(human_input, response)]

        with gr.Blocks() as demo:
            gr.Markdown("## Chat bot demo")
            with gr.Tabs():
                with gr.TabItem("Chat"):
                    chatbot = gr.Chatbot(height=600)
                    message = gr.Textbox(placeholder="Type your message here...")
                    message.submit(chat, [message, chatbot], [message, chatbot])
        demo.queue().launch(share=share)


In [ ]:
memory = FixedWindowLengthMemoryConstructor(
    system_message="Cuộc trò chuyện giữa con người và trợ lý AI.\n",
    human_symbol="[|Con người|]",
    ai_symbol="[|AI|]",
    window_length=1,
)
hcc = HCCApp(
    gen_func=gen_func,
    retrieve_func=retrieve_func,
    intent_cls_func=intent_cls,
    memory=memory
)

In [ ]:
hcc.render()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://56fc535f944187d1ee.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
while True:
    ip = input('Nhập câu hỏi: ')
    answer = hcc.answer(ip)
    print(answer)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (1830 > 1024). Running this sequence through the model will result in indexing errors


### Generated in 5.367837 seconds ###

### Generated in 44.492384 seconds ###

### Generated in 45.482689 seconds ###



KeyboardInterrupt: ignored

In [ ]:
retrieve_func('thủ tục đăng ký khai sinh')

'Mã thủ tục: 1.001193\nSố quyết định: 528/QĐ-BTP\nTên thủ tục: Thủ tục đăng ký khai sinh\nCấp thực hiện: Cấp Xã\nLoại thủ tục: TTHC được luật giao quy định chi tiết\nLĩnh vực: Hộ tịch\nTrình tự thực hiện: - Nếu lựa chọn hình thức nộp hồ sơ trực tiếp, người có yêu cầu đăng ký khai sinh nộp hồ sơ đăng ký khai sinh tại Bộ phận một cửa của UBND cấp xã có thẩm quyền; nộp lệ phí nếu thuộc trường hợp phải nộp lệ phí đăng ký khai sinh; nộp phí cấp bản sao Giấy khai sinh nếu có yêu cầu cấp bản sao Giấy khai sinh.- Nếu lựa chọn hình thức nộp hồ sơ trực tuyến, người có yêu cầu đăng ký khai sinh truy cập Cổng dịch vụ công quốc gia hoặc Cổng dịch vụ công cấp tỉnh, đăng ký tài khoản (nếu chưa có tài khoản), xác thực người dùng theo hướng dẫn, đăng nhập vào hệ thống, xác định đúng Ủy ban nhân dân cấp xã có thẩm quyền.Người có yêu cầu đăng ký khai sinh trực tuyến cung cấp thông tin theo biểu mẫu điện tử tương tác đăng ký khai sinh (cung cấp trên Cổng dịch vụ công), đính kèm bản chụp hoặc bản sao điện 

In [ ]:
intent_cls('thủ tục đăng ký khai sinh')

'hcc'

In [ ]:
print(hcc.memory.get_full_conversation())

Cuộc trò chuyện giữa con người và trợ lý AI.
[|Con người|] thủ tục đăng ký khai sinh bao gồm những trình tự nào?
[|AI|] Thủ tục đăng ký khai sinh bao gồm các bước sau:
1. Trường hợp hồ sơ đầy đủ, hợp lệ thì tiếp nhận hồ sơ; nếu tiếp nhận hồ sơ sau 15 giờ thì có Phiếu hẹn, trả kết quả cho người có yêu cầu trong ngày làm việc tiếp theo (nếu người có yêu cầu lựa chọn hình thức nộp hồ sơ trực tiếp) hoặc gửi ngay Phiếu hẹn, trả kết quả qua thư điện tử hoặc gửi tin nhắn hẹn trả kết quả qua điện thoại di động cho người có yêu cầu (nếu người có yêu cầu lựa chọn hình thức nộp hồ sơ trực tuyến).
2. Cán bộ tiếp nhận hồ sơ tại Bộ phận một cửa có trách nhiệm kiểm tra tính chính xác, đầy đủ, thống nhất, hợp lệ của hồ sơ.(i) Trường hợp hồ sơ đầy đủ, hợp lệ thì tiếp nhận hồ sơ; nếu tiếp nhận hồ sơ sau 15 giờ thì có Phiếu hẹn, trả kết quả cho người có yêu cầu trong ngày làm việc tiếp theo (nếu người có yêu cầu lựa chọn hình thức nộp hồ sơ trực tiếp) hoặc gửi ngay Phiếu hẹn, trả kết quả qua thư điện tử 